# Pre-process matrix

In [8]:
!pip install --upgrade seaborn

Using pip 23.1.2 from /home/jupyter-rqu4/.local/lib/python3.9/site-packages/pip (python 3.9)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 15.4 MB/s eta 0:00:00


In [3]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os.path
import matplotlib
import seaborn as sns
from scipy.stats import pearsonr as corr
from pygam import LinearGAM

In [4]:
# Skani output lower tiangle tab-dilimitedmatrix
ani_file='/scicomp/home-pure/rqu4/PROJECTS/GaTech/eukANI/skani.out'

c=0
ani_table = {'genome1': [], 'genome2': [], 'ANI': []}
genomes_list = list()

#ani_df = Dataframe.pd()

# Open matrix file
with open(ani_file, "r") as ani_matrix:
    for i, line in enumerate(ani_matrix):
        # Skip first line
        if c == 0:
            c +=1
            continue
            
        # if c > 10:
        #     break
        # else:
        #     c +=1
            
        row = line.strip().split("\t")
        name = os.path.basename(row[0]).strip(".fasta")
        
        genomes_list.append(name)
        
        for i2, ani_val in enumerate(row[1:]):
            # skip ANI = 0
            if ani_val != '0.00':
                ani_table['genome1'].append(min(name, genomes_list[i2]))
                ani_table['genome2'].append(max(name, genomes_list[i2]))
                ani_table['ANI'].append(ani_val)

#Make dataframe
ani_df = pd.DataFrame(ani_table)

In [5]:
print(ani_df.head())
print(ani_df.shape)

                            genome1                           genome2    ANI
0  GCF_000005845.2_ASM584v2_genomic  GCF_000006925.2_ASM692v2_genomic  98.13
1  GCF_000005845.2_ASM584v2_genomic  GCF_000007405.1_ASM740v1_genomic  98.11
2  GCF_000006925.2_ASM692v2_genomic  GCF_000007405.1_ASM740v1_genomic  99.91
3  GCF_000005845.2_ASM584v2_genomic  GCF_000007445.1_ASM744v1_genomic  96.93
4  GCF_000006925.2_ASM692v2_genomic  GCF_000007445.1_ASM744v1_genomic  96.58
(18331985, 3)


## Transposed original report

`tail -n +2 'skani.out.af' | datamash transpose --no-strict > skani.out.af.transposed`

## Further clean up
```
sed -i 's/\.fasta//g' skani.out.af.transposed
sed -i 's/assemblies\///g' skani.out.af.transposed
```

In [85]:
# import gc
# gc.collect()

In [6]:
# kani AF output every line is tab-seperated 1vall
af_file = '/scicomp/home-pure/rqu4/PROJECTS/GaTech/eukANI/skani.out.af.transposed'

genomes_list = list()
af_table = {'genome1': [], 'genome2': [], 'AF': []}


with open(af_file, "r") as af_matrix:
    for line_num, line in enumerate(af_matrix):
        
        row = line.strip().split("\t")
        
        if line_num == 0:
            
            for name in row:
                genomes_list.append(name)
                
        else:
            for i, af in enumerate(row):
                af_table['genome1'].append(min(genomes_list[i], genomes_list[line_num-1]))
                af_table['genome2'].append(max(genomes_list[i], genomes_list[line_num-1]))
                af_table['AF'].append(af)
        

#Make dataframe
af_df = pd.DataFrame(af_table)

In [7]:
print(af_df.head())
print(af_df.shape)

                            genome1                           genome2      AF
0  GCF_000005845.2_ASM584v2_genomic  GCF_000005845.2_ASM584v2_genomic  100.00
1  GCF_000005845.2_ASM584v2_genomic  GCF_000006925.2_ASM692v2_genomic   78.49
2  GCF_000005845.2_ASM584v2_genomic  GCF_000006945.2_ASM694v2_genomic    0.00
3  GCF_000005845.2_ASM584v2_genomic  GCF_000007405.1_ASM740v1_genomic   82.67
4  GCF_000005845.2_ASM584v2_genomic  GCF_000007445.1_ASM744v1_genomic   73.81
(119552356, 3)


In [8]:
df_merged = pd.merge(ani_df, af_df, on=['genome1', 'genome2'], how='left')
df_merged.head

<bound method NDFrame.head of                                                 genome1  \
0                      GCF_000005845.2_ASM584v2_genomic   
1                      GCF_000005845.2_ASM584v2_genomic   
2                      GCF_000005845.2_ASM584v2_genomic   
3                      GCF_000005845.2_ASM584v2_genomic   
4                      GCF_000006925.2_ASM692v2_genomic   
...                                                 ...   
36663965  GCF_964200035.1_368_BP0078U_scaffolds_genomic   
36663966  GCF_964200045.1_366_BP0076U_scaffolds_genomic   
36663967  GCF_964200045.1_366_BP0076U_scaffolds_genomic   
36663968             GCF_964290865.1_23S01245-7_genomic   
36663969             GCF_964290865.1_23S01245-7_genomic   

                                   genome2    ANI     AF  
0         GCF_000006925.2_ASM692v2_genomic  98.13  78.49  
1         GCF_000006925.2_ASM692v2_genomic  98.13  81.66  
2         GCF_000007405.1_ASM740v1_genomic  98.11  82.67  
3         GCF_000007405.1

In [11]:
df_merged.to_csv('/scicomp/home-pure/rqu4/PROJECTS/GaTech/eukANI/ANI_AF_combined.tab', sep='\t', index=False)

In [1]:
import gc
gc.collect()

35

In [4]:
df_merged = pd.read_csv('/scicomp/home-pure/rqu4/PROJECTS/GaTech/FCGR_classifier/combined.tab', sep='\t')



In [16]:
df_merged.head()

,genome1,genome2,ANI,AF,GEN1_genus,GEN2_genus,GEN1_species,GEN2_species
0,GCF_000005845.2_ASM584v2_genomic,GCF_000006925.2_ASM692v2_genomic,98.13,78.49,NaN,NaN,NaN,NaN
1,GCF_000005845.2_ASM584v2_genomic,GCF_000006925.2_ASM692v2_genomic,98.13,81.66,NaN,NaN,NaN,NaN
2,GCF_000005845.2_ASM584v2_genomic,GCF_000007405.1_ASM740v1_genomic,98.11,82.67,NaN,NaN,NaN,NaN
3,GCF_000005845.2_ASM584v2_genomic,GCF_000007405.1_ASM740v1_genomic,98.11,81.92,NaN,NaN,NaN,NaN
4,GCF_000006925.2_ASM692v2_genomic,GCF_000007405.1_ASM740v1_genomic,99.91,98.26,NaN,NaN,NaN,NaN


In [7]:
df_tax = pd.read_csv('/scicomp/home-pure/rqu4/PROJECTS/GaTech/FCGR_classifier/metadata_tax_assignments.csv', sep=',')

In [13]:
df_tax['genus'] = df_tax['organism_organism_name'].str.split('_').str[0]
df_tax['species'] = df_tax['organism_organism_name'].str.split('_').str[1]
df_tax['species'] = df_tax['species'].replace("sp.", np.nan)
df_tax.head()


,accession,organism_organism_name,genus,species
0,GCF_001461805.1,Enterobacter_lignolyticus,Enterobacter,lignolyticus
1,GCA_001461805.1,Enterobacter_lignolyticus,Enterobacter,lignolyticus
2,GCF_000164865.1,Enterobacter_lignolyticus_SCF1,Enterobacter,lignolyticus
3,GCA_000164865.1,Enterobacter_lignolyticus_SCF1,Enterobacter,lignolyticus
4,GCF_001856865.2,Kluyvera_intestini,Kluyvera,intestini


In [15]:
genus_assignments = df_tax.set_index('accession')['genus']
species_assignments = df_tax.set_index('accession')['species']

# Use map to create the new column
df_merged['GEN1_genus'] = df_merged['genome1'].map(genus_assignments)
df_merged['GEN2_genus'] = df_merged['genome2'].map(genus_assignments)
df_merged['GEN1_species'] = df_merged['genome1'].map(species_assignments)
df_merged['GEN2_species'] = df_merged['genome2'].map(species_assignments)


df_merged.head()

,genome1,genome2,ANI,AF,GEN1_genus,GEN2_genus,GEN1_species,GEN2_species
0,GCF_000005845.2_ASM584v2_genomic,GCF_000006925.2_ASM692v2_genomic,98.13,78.49,NaN,NaN,NaN,NaN
1,GCF_000005845.2_ASM584v2_genomic,GCF_000006925.2_ASM692v2_genomic,98.13,81.66,NaN,NaN,NaN,NaN
2,GCF_000005845.2_ASM584v2_genomic,GCF_000007405.1_ASM740v1_genomic,98.11,82.67,NaN,NaN,NaN,NaN
3,GCF_000005845.2_ASM584v2_genomic,GCF_000007405.1_ASM740v1_genomic,98.11,81.92,NaN,NaN,NaN,NaN
4,GCF_000006925.2_ASM692v2_genomic,GCF_000007405.1_ASM740v1_genomic,99.91,98.26,NaN,NaN,NaN,NaN
